![QuantConnect Logo](https://cdn.quantconnect.com/web/i/icon.png)
<hr>

# Strategy 1: Buy and hold

Buy and hold generates profit from the risk premia of market risk exposition.

A buy and hold strategy of 1 instrument is the most simple strategy. It requires several considerations that will be touched on in this notebook

## Data aquisition

In [2]:
from datetime import datetime
from QuantConnect import Resolution
from QuantConnect.Securities import Futures
from helper import calculate_percentage_returns, calculate_return_in_usd, calculate_return_in_mxn, calculate_transaction_costs, expiration_dates


In [4]:
# Initialize QuantBook
qb = QuantBook()

# Add Future Subscriptions for ES and MES with Daily resolution
es_future = qb.AddFuture(Futures.Indices.SP500EMini, Resolution.Daily)
mes_future = qb.AddFuture(Futures.Indices.MicroSP500EMini, Resolution.Daily)

# Add Forex Subscription for USD/MXN with Daily resolution
fx_pair = qb.AddForex("USDMXN").symbol


# Set Historical Data Range
start_date = datetime(2007, 1, 1)
end_date = datetime.now()

# Request Historical Data for ES and MES futures
es_history = qb.History([es_future.Symbol], start_date, end_date, Resolution.Daily)
mes_history = qb.History([mes_future.Symbol], start_date, end_date, Resolution.Daily)

# Request Historical Data for USD/MXN FX pair
fx_history = qb.History([fx_pair], start_date, end_date, Resolution.Daily)

# Save data to Object Store (optional)
# qb.ObjectStore.Save("ES_Future_Data", es_history.to_csv(index=False))
# qb.ObjectStore.Save("MES_Future_Data", mes_history.to_csv(index=False))

es_history.reset_index(level=0, drop=True, inplace=True)
mes_history.reset_index(level=0, drop=True, inplace=True)
# Reset index for the historical data

# Display the first few rows of the historical data
print("ES Future Data:")
print(es_history.head())

print("\nMES Future Data:")
print(mes_history.head())

# Display the first few rows of the historical data
print("\nUSD/MXN FX Data:")
print(fx_history.head())


In [5]:
print(es_history.index)
print(mes_history.index)
print(fx_history.index)

## Multipliers and tick size

Each contract may have different multipliers. That means, that a 1$ move is equal to a 50$ change in returns for each contract one is exposed to. We will create a dictionary with the relevant information to calculate the returns down the road.

### Compute Functions

In [6]:
instruments = {
    'ES': {
        'multiplier': 50,
        'tick_value': 0.25,
        'minimum_fluctuation': 12.50,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['ES']
    },
    'MES': {
        'multiplier': 5,
        'tick_value': 0.25,
        'minimum_fluctuation': 1.25,
        'spread': 0.25,  # Typical spread
        'commission': 2.50,  # Typical commission per contract
        'expiration_dates': expiration_dates['MES']
    }
}


In [7]:
def simulate_buy_and_hold(history_df, symbol, fx_rates_df):
    """
    Simulate buy-and-hold strategy. Takes historical price DataFrame and symbol.
    Returns net total return after accounting for transaction costs.
    """
    params = instruments[symbol]
    initial_price = history_df['close'].iloc[0]
    final_price = history_df['close'].iloc[-1]

    total_return_in_usd = calculate_return_in_usd(symbol, initial_price, final_price, instruments)
    total_return_in_mxn = calculate_return_in_mxn(total_return_in_usd, fx_rates_df)

    # Calcular costos de transacción
    transaction_costs = calculate_transaction_costs(params, history_df, start_date, end_date)

    # Ajustar retornos por costos de transacción
    net_return_in_usd = total_return_in_usd - transaction_costs
    net_return_in_mxn = calculate_return_in_mxn(net_return_in_usd, fx_rates_df)

    return {
        "return_in_points": final_price - initial_price,
        "return_in_usd": net_return_in_usd,
        "return_in_mxn": net_return_in_mxn
    }


In [8]:

# Example usage
es_returns = simulate_buy_and_hold(es_history, 'ES', fx_history)
mes_returns = simulate_buy_and_hold(mes_history, 'MES', fx_history)

print("\nES Returns:")
print(es_returns)

print("\nMES Returns:")
print(mes_returns)
